In [5]:
import numpy as np
from scipy.io import loadmat
from scipy.stats import gamma
from scipy.special import digamma, loggamma
import os
import sys
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)
from datasets import inputs, sf_inputs_discrete
from utils import get_field_metrics

import configs

# Arguments

In [3]:
iters = 3                                                                      
gammas = [0.4, 0.6, 0.8]                          

# Integer sigmas                                                               
spatial_sigmas = [0.0, 1.0, 2.0, 3.0]                                          
sparsity_range = [[0.001, 0.2], [0.001, 0.1], [0.001, 0.04], [0.001, 0.023]]   

# Other sigmas                                                                 
spatial_sigmas.extend([0.25, 0.5, 1.25, 1.5, 1.75, 2.25, 2.5, 2.75, 3.25])     
sparsity_range.extend([                                                        
    [0.001, 0.19], # 0.25                                                      
    [0.001, 0.15], # 0.5                                                       
    [0.001, 0.09], # 1.25                                                      
    [0.001, 0.05], # 1.5                                                       
    [0.001, 0.045], # 1.75                                                     
    [0.001, 0.037], # 2.25                                                     
    [0.001, 0.03], # 2.5                                                       
    [0.001, 0.025], # 2.75                                                     
    [0.001, 0.021], # 3.25                                                     
    ])                                                                         

num_states = 14*14                                                             
num_steps = 5001

In [4]:
args = []                                                                   
for gamma in gammas:                                                        
    for idx, spatial_sigma in enumerate(spatial_sigmas):                    
        _range = sparsity_range[idx]                                        
        sparsity_ps = np.linspace(_range[0], _range[1], num=20, endpoint=True)
        for sparsity_p in sparsity_ps:                                      
            args.append([gamma, spatial_sigma, sparsity_p]) 

# Helper functions

In [ ]:
def get_firing_field(raw_ff, arena_length=14, window_ratio=0.12, normalize=False             
    ):                                                                             
    """                                                                            
    The window size (area*0.12) is a ratio taken from Payne & Aronov 2021.         
                                                                                   
    Args:                                                                          
        xs: (frames,) array of x locations                                         
        ys: (frames,) array of y locations                                         
        activity: (frames,) array of activity of one neuron                        
        window_ratio: Hamming window size; default ratio from Payne 2021           
        arena_length: length of square arena                                       
    """                                                                                                                       
                                                                                   
    # Smooth field                                                                 
    window = int(np.sqrt(arena_length*arena_length*window_ratio))                  
    window = window + 1 if window%2 == 0 else window # Must be odd number          
    kernel = np.outer(hamming(window), hamming(window))                            
    firing_field = convolve(raw_ff, kernel)                                  
                                                                                   
    # Normalize, if desired                                                        
    if normalize:                                                                  
        max_rate = firing_field.max()                                              
        if max_rate != 0:                                                          
            firing_field = firing_field/firing_field.max()                         
                                                                                   
    return firing_field  

In [ ]:
def get_field_metrics(feature_map):                                                                      
    """                                                                         
    Args:                                                                       
        activity: (frames, neurs) array of activity of all neurons              
        arena_length: length of square arena                                    
    """                                                                         
                                                                                                                                                                                                    
    fieldsizes = []                                                             
    nfields = []                                                                
                                                                                                                                     
    # Go over each neuron                                                       
    for neur in np.arange(feature_map.shape[1]):           
        ff = feature_map[:, neur].reshape((14,14))
        field, nan_idxs = get_firing_field(xs, ys, activity[:, neur], arena_length)
        field_mask = np.zeros(field.shape)                                  
        for _ in range(nshuffles):                                          
            shuffled_field, _ = get_firing_field(                           
                xs, ys, circular(activity[:, neur]), arena_length           
                )                                                           
            field_mask += (field > shuffled_field)                          
        zz = np.copy(field_mask)                                            
        field_mask = field_mask > 0.99*nshuffles                            
        if save_field_info:                                                 
            field_infos.append((field, field_mask, nan_idxs)) 

# Collect metrics

In [9]:
gammas = []
spatial_sigmas = []
sparsities = []
fieldsize_kls = []
fieldnum_kls = []
for arg in args:
    _gamma, _spatial_sigma, _sparsity_p = arg                                  
    dataset = sf_inputs_discrete.Sim2DWalk                                  
    feature_maker_kwargs = {                                                
        'feature_dim': num_states, 'feature_type': 'correlated_distributed',
        'feature_vals_p': [1-_sparsity_p, _sparsity_p],                       
        'spatial_sigma': _spatial_sigma                                      
        }                                             
    dataset_config = {                                                      
        'num_steps': num_steps,
        'feature_maker_kwargs': feature_maker_kwargs,
        'num_states': num_states                                            
        }
    dset = dataset(**dataset_config)   
    features = dset.feature_maker.feature_map.shape # (features, state)
    fieldsize_kls.append()
    fieldnum_kls.append()
    sparsities.append(dset.feature_maker.post_smooth_sparsity)

TypeError: unsupported operand type(s) for *: 'gamma_gen' and 'float'

# Plotting Helper Functions

# Make heatmaps